In [1]:
import  pandas as pd
import time, os
from random import randint
import geopy.distance
import geopy
from geopy.geocoders import Nominatim

# Comparando coordenadas endereço com coordenadas cidade usando geopy

In [2]:
dados = pd.read_csv('dados/primeira_infancia_tratado.csv')

In [3]:
dados

,ID_FAMILIA_CADUNICO,CODG_FAMILIAR,DESC_STAT_CADASTRO,INDI_CADASTRO_VALIDO,DESC_CONDICAO_CADASTRO,DATA_CADASTRAM,DATA_ATUALZ_FAMILIA,CODG_IBGE_MUN_RESIDE,NOME_LOCALIDADE,NUMR_CEP_LOGRADOURO,...,Municípios.NOME_MUNICIPIO,Municípios.CODG_CEP_GENERICO,Municípios.TIPO_LOCALIDADE,Municípios.INFO_LATITUDE,Municípios.INFO_LONGITUDE,ENDERECO,lat_cidade,lon_cidade,LAT,LOG
0,3229,75858614,Cadastro,N,Atualizado,"sexta-feira, 5 de abril de 2002","quinta-feira, 26 de abril de 2018",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 16, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.090816,-48.508371
1,3532,79167101,Excluído,N,Desatualizado,"quarta-feira, 10 de abril de 2002","sexta-feira, 19 de junho de 2015",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 16, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.090816,-48.508371
2,4233,496751336,Excluído,N,Desatualizado,"terça-feira, 5 de novembro de 2002","sábado, 23 de março de 2013",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 92, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.082136,-48.507431
3,4250,496751840,Cadastro,N,Atualizado,"terça-feira, 5 de novembro de 2002","sexta-feira, 25 de novembro de 2016",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 8 A, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.095424,-48.500624
4,4443,575815094,Cadastro,S,Desatualizado,"segunda-feira, 6 de janeiro de 2003","quinta-feira, 12 de fevereiro de 2015",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 90, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.083491,-48.507556
5,4583,575820179,Cadastro,N,Atualizado,"segunda-feira, 6 de janeiro de 2003","terça-feira, 18 de julho de 2017",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 102, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.082136,-48.507431
6,4785,575827181,Cadastro,N,Atualizado,"segunda-feira, 6 de janeiro de 2003","terça-feira, 8 de novembro de 2016",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 120, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.082136,-48.507431
7,4854,575829397,Cadastro,N,Atualizado,"segunda-feira, 6 de janeiro de 2003","terça-feira, 14 de agosto de 2018",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 84, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.685768,-49.254747
8,5225,575847450,Cadastro,N,Atualizado,"segunda-feira, 6 de janeiro de 2003","terça-feira, 20 de junho de 2017",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 85, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.082136,-48.507431
9,6807,1787678598,Excluído,N,Desatualizado,"terça-feira, 25 de julho de 2006","segunda-feira, 18 de março de 2013",5200308,CENTRO,72930000.0,...,ALEXANIA,72930000,M,"16° 04' 56""","48° 30' 26""","RUA 14, SN, CENTRO, ALEXANIA, GOIAS",-16.0834,-48.5076,-16.091952,-48.505433


In [4]:
df1 = pd.DataFrame(columns=['ID_FAMILIA_CADUNICO','DESVIO'])
retornos = 0
retate1km = 0
for index, row in dados.iterrows():
    try:
        
        if row["LAT"] != "None" and row["LAT"] != "ERROR" and row["lat_cidade"] != "None" and row["lat_cidade"] != "ERROR":
            retornos=retornos + 1
            
            p1 = (float(row["LAT"]),float(row["LOG"]))
            p2 = (float(row["lat_cidade"]),float(row["lon_cidade"]))

            distkm = geopy.distance.distance(p1, p2).km
            
            df1.loc[index] = [row['ID_FAMILIA_CADUNICO'], distkm]
            
            if distkm <= 1:
                retate1km = retate1km + 1
            #print("%.3f" %(geopy.distance.distance(p1, p2).km))
    except:
    #else:
        df1.loc[index] = [row['ID_FAMILIA_CADUNICO'],"Nan"]
        #print(None)


In [5]:
df1.describe().round(2)

,ID_FAMILIA_CADUNICO
count,5469.00
mean,5106.11
std,2665.04
min,9.00
25%,3087.00
50%,5120.00
75%,7079.00
max,10000.00


In [6]:
novo_dados = pd.concat([dados, df1['DESVIO']], axis=1)

In [7]:
novo_dados.isnull().sum()

ID_FAMILIA_CADUNICO                  0
CODG_FAMILIAR                        0
DESC_STAT_CADASTRO                   0
INDI_CADASTRO_VALIDO                 0
DESC_CONDICAO_CADASTRO              85
DATA_CADASTRAM                       0
DATA_ATUALZ_FAMILIA                  0
CODG_IBGE_MUN_RESIDE                 0
NOME_LOCALIDADE                    103
NUMR_CEP_LOGRADOURO                103
INFO_REFERENCIA_LOCAL             4145
CODG_UNIDADE_TERRITAL             3327
NOME_UNIDADE_TERRITAL             3327
VALR_RENDA_MEDIA                     0
INDI_RECEBE_BOLSA_FAMILIA            0
INDI_BANHEIRO_DOMICILIO            202
TIPO_DESTINO_LIXO                  202
TIPO_MATERIAL_PISO                 202
TIPO_MATERIAL_PAREDE_EXTERNA       202
INDI_AGUA_CANALIZADA               202
TIPO_CALCAMENTO                    305
TIPO_ESPECIE_DOMICILIO              63
QTDE_CRIANCAS_ATE_3ANOS              0
QTDE_CRIANCAS_ATE_6ANOS              0
QTDE_MAIOR18_NAO_SABE_LER            0
QTDE_ENTRE6E17_NAO_FREQ_E

## Alterando os valores com erro por Lat, Lon da Cidade

In [8]:
novo_dados['aproximação'] = novo_dados['DESVIO'].apply(lambda x: 'Erro' if float(x) >= 10.0 else 'ok')

estatistica = novo_dados['aproximação'] == 'Erro'
porcentagem_erro = estatistica.sum() / len(novo_dados)*100
print('Valores fora do perimetro: '+ str(porcentagem_erro.round(2)) +'%')

Valores fora do perimetro: 17.9%


In [9]:
dados_desvio = novo_dados.to_csv('dados/dados_primeira_infancia_com_desvio.csv', index = None, header=True)

### Filtros 

In [10]:
def filrando_por_cidade(nome_cidade:'recebe o nome do municipo'):
    selecao = (novo_dados['Municípios.NOME_MUNICIPIO'] == nome_cidade)
    df1 = novo_dados[selecao]
    d = df1['aproximação'].value_counts()
    return d

In [11]:
filrando_por_cidade('ALEXANIA')

ok      1023
Erro     404
Name: aproximação, dtype: int64

In [12]:
filrando_por_cidade('ABADIA DE GOIAS')

ok      1187
Erro     143
Name: aproximação, dtype: int64

In [13]:
selecao = (novo_dados['Municípios.NOME_MUNICIPIO'] == 'ALEXANIA')
df1 = novo_dados[selecao]

In [14]:
df1['aproximação'].value_counts()

ok      1023
Erro     404
Name: aproximação, dtype: int64

In [15]:
filrando_por_cidade('ALEXANIA')

ok      1023
Erro     404
Name: aproximação, dtype: int64

In [16]:
import seaborn as sns
df = novo_dados['lat_cidade']
x =  novo_dados['DESVIO']
 
sns.boxplot(data=df, saturation=1, orient = 'h')
#sns.plt.show()

